In [4]:
import os
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

def search_and_save_articles():
    data = input('Enter your search keyword: ')

    url = f'https://scholar.google.com/scholar?q={data}'
    response = requests.get(url)

    if response.status_code != 200:
        print('Failed to retrieve search results. Please check your internet connection.')
        return

    soup = BeautifulSoup(response.text, "html.parser")

    results = soup.find_all('div', {'class': 'gs_ri'})

    if not results:
        print('No articles found.')
        return

    article_data = {
        'No': [],
        'Judul': [],
        'Author': [],
        'Tahun': [],
        'Nama Jurnal': [],
        'Link': [],
        'Type': [],
        'Proses Pencarian': []  # Kolom baru untuk menyimpan tema pencarian
    }

    for i, result in enumerate(results):
        title_elem = result.find('h3', {'class': 'gs_rt'})
        author_elem = result.find('div', {'class': 'gs_a'})
        link_elem = result.find('a')

        if title_elem and author_elem and link_elem:
            title = title_elem.get_text()
            author_text = author_elem.get_text()
            link = link_elem.get('href')

            # Menggunakan ekspresi reguler untuk mencocokkan author, tahun, dan kategori
            match = re.search(r'^(.*?)(?:\s*-?\s*(\d{4}))?(?:\s*-\s*(.*?))?\s*$', author_text)
            if match:
                author = match.group(1).strip()
                year = match.group(2)
                category = match.group(3).strip() if match.group(3) else ''

                # Cek apakah artikel adalah prosiding berdasarkan tautan atau kata kunci dalam judul atau tautan
                seminar = 'Prosiding' if 'proceedings' in link.lower() or 'conference' in title.lower() or 'symposium' in title.lower() else 'Jurnal'

                # Cek apakah kata "prosiding" muncul dalam nama jurnal
                if 'prosiding' in category.lower():
                    seminar = 'Prosiding'

                # Cek apakah kata "book" atau "books" muncul dalam nama jurnal
                if 'book' in category.lower() or 'books' in category.lower():
                    seminar = 'Buku'

                # Cek apakah kategori mengandung empat digit angka
                year_match = re.search(r'\d{4}', category)
                if year_match:
                    # Jika ada empat digit angka dalam kategori, ambil sebagai tahun
                    year = year_match.group()
                    # Hapus tahun dari kategori
                    category = category.replace(year, '').strip()

            # Menyimpan tema pencarian dalam kolom "Proses Pencarian"
            article_data['Proses Pencarian'].append(data)

            article_data['No'].append(i + 1)
            article_data['Judul'].append(title)
            article_data['Author'].append(author)
            article_data['Tahun'].append(year)
            article_data['Nama Jurnal'].append(category)
            article_data['Link'].append(link)
            article_data['Type'].append(seminar)

    df = pd.DataFrame(article_data)
    df.to_csv('artikel_ujiT.csv', index=False)
    print('Articles saved to articles.csv')

if __name__ == '__main__':
    search_and_save_articles()


Articles saved to articles.csv
